In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, SpatialDropout1D, LSTM, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Activation, Dropout
# import LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from matplotlib import pyplot as plt

from imblearn.over_sampling import SMOTE
from collections import Counter


/var/folders/4_/8jk7rgt141nghfj9gsw4_thm0000gn/T/ipykernel_72936/370164429.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-17 15:25:13.397130: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
#pip install gensim


In [20]:
from gensim.models import Word2Vec


In [4]:
def graph_plots(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.savefig(string + '.png')
    plt.show()

In [5]:
df = pd.read_csv('cleaned_dataset_1000.csv')
df.columns

Index(['type', 'posts', 'cleaned_posts'], dtype='object')

In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['cleaned_posts'])
tokenizer.word_index # Get our learned vocabulary
word_index = tokenizer.word_index

VOCAB_SIZE = len(word_index)+1 # Total words

In [7]:
X = tokenizer.texts_to_sequences(df['cleaned_posts'])
MAX_SEQ_LENGTH = max(len(seq) for seq in X)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen= MAX_SEQ_LENGTH) # Pad the sequence to the same length to make it uniform

In [12]:
labels = LabelEncoder().fit_transform(df.type)


In [13]:
smote = SMOTE(random_state=42, k_neighbors=1)
X_res, y_res = smote.fit_resample(X,labels)
X_res.shape, y_res.shape

((3792, 892), (3792,))

## Word2Vec

In [24]:
word2vec_model = Word2Vec(X_res, vector_size=100, window=5, min_count=1, workers=4)


Exception in thread Exception in thread Thread-20:
Traceback (most recent call last):
  File "/Users/zainanasalma/opt/anaconda3/envs/cs229/lib/python3.9/threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-21:
Traceback (most recent call last):
  File "/Users/zainanasalma/opt/anaconda3/envs/cs229/lib/python3.9/threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-22:
Traceback (most recent call last):
  File "/Users/zainanasalma/opt/anaconda3/envs/cs229/lib/python3.9/threading.py", line 980, in _bootstrap_inner
Thread-19:
Traceback (most recent call last):
  File "/Users/zainanasalma/opt/anaconda3/envs/cs229/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/Users/zainanasalma/opt/anaconda3/envs/cs229/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    self.run()
  File "/Users/zainanasalma/opt/anaconda3/envs/cs229/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 761, in run_closur

In [1]:
model_path = "/Users/zainanasalma/Desktop/stanford/courses/CS229/mbti-cs229/Word2vec_model.model"
word2vec_model.save(model_path)

NameError: name 'word2vec_model' is not defined

In [ ]:
embedding_size = 100
embedding_matrix = np.zeros((VOCAB_SIZE, embedding_size))

for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_vector = word2vec_model.wv[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [16]:
y_one_hot = pd.get_dummies(y_res)
n_classes = y_one_hot.shape[1]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_one_hot, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
rate = 0.1
model = Sequential()
best_model.add(Embedding(VOCAB_SIZE, embedding_size, input_length=MAX_SEQ_LENGTH, weights=[embedding_matrix], trainable=False))
best_model.add(SimpleRNN(100))
best_model.add(Dropout(rate))
best_model.add(Dense(n_classes, activation='softmax'))
best_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])